In [ ]:
# colab: runtime -> change runtime type -> GPU
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'
!pip install opencv-python matplotlib numpy trimesh


Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-3wq4y1pb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-3wq4y1pb
  Running command git checkout -q 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA available: True
Device name: NVIDIA A100-SXM4-40GB


In [ ]:
import os, json, cv2, torch, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    look_at_view_transform, OpenGLPerspectiveCameras, PointLights,
    RasterizationSettings, MeshRasterizer, MeshRenderer, HardPhongShader
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 384
default_fov_deg = 60.0
print("Using device:", device)


Using device: cuda


In [ ]:
from huggingface_hub import login
login()  # paste your token when prompted


In [ ]:
from huggingface_hub import snapshot_download

local_dir = "/content/drive/MyDrive/3DObjectReconstruction/data/shapenet"

snapshot_download(
    repo_id="ShapeNet/ShapeNetCore",
    repo_type="dataset",
    local_dir=local_dir,
    allow_patterns=[
        "03001627/1a0bc9ab92c915167ae33d942430658c/models/model_normalized.obj"
    ],
)


Fetching 0 files: 0it [00:00, ?it/s]

'/content/drive/MyDrive/3DObjectReconstruction/data/shapenet'